### 4. PLAY TENNIS CLASSIFIER IMPLEMENTATION

Cho trước dữ liệu thời tiết của 10 ngày (D1-D10,như bảng 1) hãy phát triển chương trình sử dụng mô hình phân loại Naive Bayes để dự đoán xem ngày thứ 11 (D11), AD có thể chơi tennis hay không ?

| Day         | Outlook | Temperature | Humidity | Wind   | PlayTennis |
|-------------|---------|--------------|----------|--------|------------|
| **D11**     | Sunny   | Cool         | High     | Strong | ???        |

4.1 Hoàn thiện function create_train_dateset() để tổ chức dữ liệu bảng 1 vào array 2 chiều như bên dưới.

In [3]:
# ########################
# Create data
# ########################
import numpy as np


def create_train_data():
	# your code here ************************
	data = [['Sunny', 'Hot', 'High', 'Weak', 'no'],
	        ['Sunny', 'Hot', 'High', 'Strong', 'no'],
	        ['Overcast', 'Hot', 'High', 'Weak', 'yes'],
	        ['Rain', 'Mild', 'High', 'Weak', 'yes'],
	        ['Rain', 'Cool', 'Normal', 'Weak', 'yes'],
	        ['Rain', 'Cool', 'Normal', 'Strong', 'no'],
	        ['Overcast', 'Cool', 'Normal', 'Strong', 'yes'],
	        ['Overcast', 'Mild', 'High', 'Weak', 'no'],
	        ['Sunny', 'Cool', 'Normal', 'Weak', 'yes'],
	        ['Rain', 'Mild', 'Normal', 'Weak', 'yes']]
	return np.array(data)


train_data = create_train_data()
print(train_data)



[['Sunny' 'Hot' 'High' 'Weak' 'no']
 ['Sunny' 'Hot' 'High' 'Strong' 'no']
 ['Overcast' 'Hot' 'High' 'Weak' 'yes']
 ['Rain' 'Mild' 'High' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'no']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'yes']
 ['Overcast' 'Mild' 'High' 'Weak' 'no']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'yes']]


4.2 Hoàn thiện function compute_prior_probability tính P("Play Tennis"="Yes") và tính P("Play Tennis"="No")

In [11]:
def compute_prior_probability(train_data):
	y_uniqe = ['no', 'yes']
	prior_probability = np.zeros(len(y_uniqe))
	data = train_data.copy()
	count_no = 0
	count_yes = 0
	for element in data:
		if element[-1] == y_uniqe[0]:
			count_no += 1
		if element[-1] == y_uniqe[1]:
			count_yes += 1

	prior_probability = (count_no / len(data), count_yes / len(data))
	return prior_probability


prior_probablity = compute_prior_probability(train_data)
print("P( play tennis = No) = ", prior_probablity[0])
print("P( play tennis = Yes) = ", prior_probablity[1])

P( play tennis = No) =  0.4
P( play tennis = Yes) =  0.6


4.3 Hoàn thiện function compute_conditional_probability để tính likelihood (the Probability) of A being True. Given "B" True, P(A|B))

In [29]:

def compute_conditional_probability(train_data):
	# Các giá trị nhãn duy nhất (giả định là có hai giá trị: 'no' và 'yes')
	y_unique = ['no', 'yes']
	# Danh sách để lưu xác suất có điều kiện cho mỗi đặc trưng
	conditional_probability = []
	# Danh sách để lưu các giá trị duy nhất của mỗi đặc trưng
	list_x_name = []

	# Duyệt qua từng cột của dữ liệu trừ cột cuối cùng (nhãn)
	for i in range(0, train_data.shape[1] - 1):
		# Lấy các giá trị duy nhất của đặc trưng hiện tại
		x_unique = np.unique(train_data[:, i])
		list_x_name.append(x_unique)

		# Khởi tạo ma trận để lưu xác suất có điều kiện
		# Hàng là các giá trị của nhãn, cột là các giá trị của đặc trưng
		x_conditional_probability = np.zeros((len(y_unique), len(x_unique)))

		# Duyệt qua từng giá trị nhãn
		for j in range(len(y_unique)):
			# Duyệt qua từng giá trị đặc trưng
			for k in range(len(x_unique)):
				# Đếm số lần xuất hiện của giá trị đặc trưng (x_unique[k]) khi nhãn là y_unique[j]

				x_conditional_probability[j, k] = (len(np.where((train_data[:, i] == x_unique[k])
				                                                & (train_data[:, -1] == y_unique[j]))[0])
				                                   / len(np.where(train_data[:, -1] == y_unique[j])[0]))

		# Lưu xác suất có điều kiện của đặc trưng hiện tại
		conditional_probability.append(x_conditional_probability)

	# Trả về danh sách các xác suất có điều kiện và danh sách các giá trị duy nhất của từng đặc trưng
	return conditional_probability, list_x_name


compute_conditional_probability(train_data)

([array([[0.25      , 0.25      , 0.5       ],
         [0.33333333, 0.5       , 0.16666667]]),
  array([[0.25      , 0.5       , 0.25      ],
         [0.5       , 0.16666667, 0.33333333]]),
  array([[0.75      , 0.25      ],
         [0.33333333, 0.66666667]]),
  array([[0.5       , 0.5       ],
         [0.16666667, 0.83333333]])],
 [array(['Overcast', 'Rain', 'Sunny'], dtype='<U8'),
  array(['Cool', 'Hot', 'Mild'], dtype='<U8'),
  array(['High', 'Normal'], dtype='<U8'),
  array(['Strong', 'Weak'], dtype='<U8')])

 Câu hỏi 15: Đáp dựa vào 4.3 => đáp án b

4.4 Hoàn thiện function get_index_from_value để tính trả về index tương ứng với feature name:

In [19]:
# This function is used to return the index of the feature name
def get_index_from_value(feature_name, list_features):
	return np.where(list_features == feature_name)[0][0]


train_data = create_train_data()
_, list_x_name = compute_conditional_probability(train_data)
outlook = list_x_name[0]

i1 = get_index_from_value("Overcast", outlook)
i2 = get_index_from_value("Rain", outlook)
i3 = get_index_from_value("Sunny", outlook)

print(i1, i2, i3)

0 1 2


Câu hỏi 16 => đáp n c) 0 1 2

In [26]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
# Compute P(" Outlook "=" Sunny "| Play Tennis "=" Yes ")
x1 = get_index_from_value("Sunny", list_x_name[0])

print("P( ’ Outlook ’= ’ Sunny ’| Play Tennis ’= ’Yes ’) = ", np.round(conditional_probability[0][1, x1], 2))

2
0.0
P( ’ Outlook ’= ’ Sunny ’| Play Tennis ’= ’Yes ’) =  0.17


Câu hỏi 17 => d) P('Outlook' ='Sunny'|'Play Tennis' = 'Yes') = 0.17

In [35]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
# Compute P(" Outlook "=" Sunny "| Play Tennis "=" No ")
x1 = get_index_from_value("Sunny", list_x_name[0])
print("P( ’ Outlook ’= ’ Sunny ’| Play Tennis ’= ’No ’) = ", np.round(conditional_probability
                                                                      [0][0, x1], 2))

P( ’ Outlook ’= ’ Sunny ’| Play Tennis ’= ’No ’) =  0.5


Câu hỏi 18 =>  a) P('Outlook' ='Sunny'|'Play Tennis' = 'No') = 0.5

In [32]:
# ##########################
# Train Naive Bayes Model
# ##########################
def train_naive_bayes(train_data):
	# Step 1: Calculate Prior Probability
	y_unique = ['no', 'yes']
	prior_probability = compute_prior_probability(train_data)

	# Step 2: Calculate Conditional Probability
	conditional_probability, list_x_name = compute_conditional_probability(train_data)

	return prior_probability, conditional_probability, list_x_name

In [33]:
# ###################
# Prediction
# ###################
def prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability):
	x1 = get_index_from_value(X[0], list_x_name[0])
	x2 = get_index_from_value(X[1], list_x_name[1])
	x3 = get_index_from_value(X[2], list_x_name[2])
	x4 = get_index_from_value(X[3], list_x_name[3])

	p0 = prior_probability[0] \
	     * conditional_probability[0][0, x1]\
	     * conditional_probability[1][0, x2]\
	     * conditional_probability[2][0, x3]\
	     * conditional_probability[3][0, x4]
	p1 =prior_probability[1] \
	     * conditional_probability[0][1, x1]\
	     * conditional_probability[1][1, x2]\
	     * conditional_probability[2][1, x3]\
	     * conditional_probability[3][1, x4]

	# your code here ***********************

	if p0 > p1:
		y_pred = 0
	else:
		y_pred = 1

	return y_pred

In [34]:
X = ['Sunny', 'Cool', 'High', 'Strong']

data = create_train_data()

prior_probability, conditional_probability, list_x_name = train_naive_bayes(data)
pred = prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability)

if (pred):
	print("Ad should go!")
else:
	print("Ad should not go!")

Ad should not go!


Câu hỏi 19: đáp án a) ad should not go